# VAR USA

Vector auto-regressive models

1. [Imports](#imports)
2. [Ingestion](#ingestion)
3. [Plotting](#plotting)
4. [Statistical tests](#stattests1)
5. [Differencing](#diff)


### VAR with First-Order Differencing
1. [Train test split - first order differencing](#traintest1)
2. [Find order p of VAR](#var_p_1)
3. [VAR(8) Model](#var8_1)
4. [Plots of first differenced predictions](#diff1_plot)
5. [Undifferencing and predicting](#undiff_1)
6. [MAPE](#mape1)
7. [Rolling forecasts](#roll1)


### VARMA
1. [VARMA](#varma)
2. [Rolling forecasts](#roll2)

<a name=imports></a>
## Imports

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings

warnings.filterwarnings("ignore")

## Results table

In [ ]:
results_columns = ['model', 'mse', 'mape', 'mae']
results_table = pd.DataFrame(columns=results_columns)

<a name=ingestion></a>
## Ingestion

In [ ]:
daily_cases_usa = pd.read_csv('../../cleaned_datasets/usa/daily_cases_usa.csv', parse_dates=['Date'])
daily_vacc_usa = pd.read_csv('../../cleaned_datasets/usa/daily_vacc_usa.csv', parse_dates=['date'])

In [ ]:
daily_cases_usa.dtypes

In [ ]:
daily_cases_usa

In [ ]:
daily_vacc_usa.dtypes

In [ ]:
daily_vacc_usa

In [ ]:
cases_vacc = daily_cases_usa.merge(daily_vacc_usa, how='outer', left_on='Date', right_on='date')
cases_vacc = cases_vacc[["Date", "Confirmed", "Total_Doses"]]
cases_vacc

In [ ]:
cases_vacc.fillna(0, inplace=True)
indexed = cases_vacc.set_index('Date')
indexed

<a name=plotting></a>
## Plot initial data

In [ ]:
def plot_subplots(indexed):
    
    if type(indexed) == pd.DataFrame:
        nrows = int(len(indexed.columns)/2)
    else:
        nrows = 1
        
    fig, axes = plt.subplots(nrows=nrows, ncols=2, dpi=120, figsize=(8,4))
    for i, ax in enumerate(axes.flatten()):
        
        if type(indexed) == list:
            # fig, ax = plt.subplots()
            # fig.set_size_inches(8, 8)

            ax.plot(indexed[0][indexed[0].columns[i]], color='blue', label = 'Train')
            ax.plot(indexed[1][indexed[1].columns[i]], color='red', label = 'Test')
            ax.legend(loc = 'best')
            # Decorations
            ax.set_title(indexed[0].columns[i])
        else:    
            data = indexed[indexed.columns[i]]
            ax.plot(data, color='blue', linewidth=1)
            # Decorations
            ax.set_title(indexed.columns[i])
        ax.xaxis.set_ticks_position('none')
        ax.yaxis.set_ticks_position('none')
        ax.spines["top"].set_alpha(0)
        ax.tick_params(labelsize=6)
    
    plt.tight_layout()
    
plot_subplots(indexed)

<a name=stattests1></a>
## Statistical tests

### Johansen co-integration test

In [ ]:
from statsmodels.tsa.vector_ar.vecm import coint_johansen

def cointegration_test(df, alpha=0.05): 
    """Perform Johanson's Cointegration Test and Report Summary"""
    out = coint_johansen(df,-1,5)
    d = {'0.90':0, '0.95':1, '0.99':2}
    traces = out.lr1
    cvts = out.cvt[:, d[str(1-alpha)]]
    def adjust(val, length= 6): return str(val).ljust(length)

    # Summary
    print('Name   ::  Test Stat > C(95%)    =>   Signif  \n', '--'*20)
    for col, trace, cvt in zip(df.columns, traces, cvts):
        print(adjust(col), ':: ', adjust(round(trace,2), 9), ">", adjust(cvt, 8), ' =>  ' , trace > cvt)

cointegration_test(indexed)

The two datasets are correlated.

### Augmented DF Test

In [ ]:
from statsmodels.tsa.stattools import adfuller

def run_dicky_fuller(ts):
  '''Function to run Augmented Dicky Fuller test on the passed time series and report the statistics from the test'''
  print("Observations of Dickey-fuller test")
  dftest = adfuller(ts,autolag='AIC')
  dfoutput=pd.Series(dftest[0:4],index=['Test Statistic','p-value','#lags used','number of observations used'])

  for key,value in dftest[4].items():
      dfoutput['critical value (%s)'%key]= value
  print(dfoutput)


# ADF Test on each column
for name, column in indexed.iteritems():
    run_dicky_fuller(column)
    print('\n')

The TS is not stationary

<a name=diff></a>
## Differencing

## First order differencing

In [ ]:
df_diff_1 = indexed.diff().dropna()
df_diff_1

In [ ]:
# ADF Test on each column
for name, column in df_diff_1.iteritems():
    run_dicky_fuller(column)
    print('\n')

In [ ]:
plot_subplots(df_diff_1)

<a name=traintest1></a>
## Train-test split

### Train-test split - first order differenced

In [ ]:
percent_90 = int(len(df_diff_1)*0.9)

train = df_diff_1.iloc[:percent_90].dropna()
test = df_diff_1.iloc[percent_90:]

plot_subplots([train, test])

<a name=var_p_1></a>
### Finding the best value of p for VAR(p)

Using PACF plot

In [ ]:
from statsmodels.graphics.tsaplots import plot_pacf, plot_acf

pacf_var_confirmed = plot_pacf(df_diff_1['Confirmed'], lags=25)

Possible values of p - 3, 6, 10

In [ ]:
def MAPE(Y_actual, Y_Predicted, title=None):
    mask = Y_actual != 0
    
    mape = np.mean(np.abs((Y_actual - Y_Predicted)/Y_actual)[mask])*100
    if title:
        print(f"MAPE of {title} is {mape[mape.index[0]]}%")
    return mape

In [ ]:
from statsmodels.tsa.statespace.varmax import VARMAX
from sklearn.metrics import mean_squared_error, mean_absolute_error

# evaluate an VARMA model for a given order (p,d,q)
def evaluate_varma_model(train, test, varma_order, column):
    
    model = VARMAX(train, order=varma_order)
    model_fit = model.fit()
    yhat = model_fit.forecast(len(test))
    mse = mean_squared_error(test[column], yhat[column])
    mape = MAPE(test[column], yhat[column])
    mae = mean_absolute_error(test[column], yhat[column])
    return mse, mape, mae

In [ ]:
# Grid search

# evaluate combinations of p and q values for an VARMA model
def evaluate_models(train, test, p_values, q_values, column):
    #dataset = dataset.astype('float32')
    best_score, best_cfg = float("inf"), None
    
    for p in p_values:
        for q in q_values:
            order = (p,q)
            try:
                mse, mape, mae = evaluate_varma_model(train, test, order, column)
                if mse < best_score:
                    best_score, best_cfg = mse, order
                
                print('VARMA%s MSE=%.3f, MAPE=%.3f, MAE=%.3f' % (order, mse, mape, mae))
            except:
                continue
    print()
    print('Best VARMA%s MSE=%.3f' % (best_cfg, best_score))

In [ ]:
evaluate_models(train, test, [3, 6, 10], [0], column='Confirmed')

In [ ]:
evaluate_models(train, test, [3, 6, 10], [0], column='Total_Doses')

Models to try: VAR(3) and VAR(10)

<a name=var8_1></a>
### VAR(3) Model

In [ ]:
model = VARMAX(train, order=(3,0))
model_fit = model.fit()
forecasted = model_fit.forecast(len(test))

df_forecast = pd.DataFrame(forecasted, index=test.index)
df_forecast.rename(columns = {'Confirmed': 'Confirmed_forecast', 'Total_Doses': 'Total_Doses_forecast'}, inplace=True)

forecasted_conf = df_forecast.join(test)[['Confirmed', 'Confirmed_forecast']]
forecasted_vacc = df_forecast.join(test)[['Total_Doses', 'Total_Doses_forecast']]

In [ ]:
model_fit.summary()

<a name=diff1_plot></a>
### Plot Predictions on Diffed

In [ ]:
def plot_fore_test(test, fore, title):
    
    fig, ax = plt.subplots()
    fig.set_size_inches(7, 7)

    ax.plot(test, color='blue', label='Test')
    ax.plot(fore, color='red', label='Forecast')
    ax.legend(loc='best')
    plt.title(title)
    plt.show()


In [ ]:
plot_fore_test(forecasted_conf[['Confirmed']], forecasted_conf[['Confirmed_forecast']], title='Daily cases')

In [ ]:
plot_fore_test(forecasted_vacc[['Total_Doses']], forecasted_vacc[['Total_Doses_forecast']], title='Daily vaccinations')

<a name=undiff_1></a>

## Un-differencing and Plotting

In [ ]:
def invert_transformation(diffed, original):
    """Revert back the differencing to get the forecast to original scale."""
    df_copy = original.copy()
    
    columns = diffed.columns
    
    for col in columns:
        # Add corresponding diff column
        df_copy[f'undiff_{col}'] = original[col]
        df_copy[f'undiff_{col}'][1:] = diffed[col]
        df_copy[f'undiff_{col}'] = df_copy[f'undiff_{col}'].cumsum()
    
        df_copy.drop(col, axis=1, inplace=True)
        
    return df_copy

In [ ]:
# Check if un-diffed df_diff_1 is the same as original

#inverted = invert_transformation(df_diff_1, indexed)
#(inverted[['undiff_Confirmed']] - indexed[['Confirmed']]).isna().sum()

In [ ]:
# Un-diff the test dataset

start_index = indexed.index.get_loc(test.index[0])-1
test_original = invert_transformation(test, indexed.iloc[start_index:])

#test_original

In [ ]:
renamed_df = df_forecast.rename(columns={'Confirmed_forecast': 'Confirmed', 'Total_Doses_forecast': 'Total_Doses'}, inplace=False)


start_index = indexed.index.get_loc(renamed_df.index[0])-1
fore_original = invert_transformation(renamed_df, indexed.iloc[start_index:])

#fore_original

In [ ]:
end_index = indexed.index.get_loc(fore_original.index[0])
train_original = indexed[:end_index]
plot_subplots([train_original, fore_original])

### Store metrics

In [ ]:
results_table = results_table.append({
    'model': 'VAR(3)',
    'mse': mean_squared_error(test_original['undiff_Confirmed'], fore_original['undiff_Confirmed']),
    'mape': MAPE(test_original['undiff_Confirmed'], fore_original['undiff_Confirmed']),
    'mae': mean_absolute_error(test_original['undiff_Confirmed'], fore_original['undiff_Confirmed'])
}, ignore_index=True)

results_table

### Plot the forecasts

In [ ]:
plot_fore_test(test_original[['undiff_Confirmed']], fore_original[['undiff_Confirmed']], title='Daily cases')

In [ ]:
plot_fore_test(test_original[['undiff_Total_Doses']], fore_original[['undiff_Total_Doses']], title='Daily vaccinations')

In [ ]:
def plot_train_test_fore(train, test, fore, title='Forecast vs Actuals', ylabel='', xlabel='Date', figpath=None):
    plt.figure(figsize=(10,5), dpi=100)
    plt.plot(train, label='training')
    plt.plot(test, label='actual')
    plt.plot(fore, label='forecast')
    fig = plt.gcf()
    
    plt.xlabel=xlabel
    plt.ylabel=ylabel

    plt.title(title)
    plt.legend(loc='upper left', fontsize=8)
    plt.show()
    
    
    if figpath is not None:
        fig.savefig(figpath, format='eps', bbox_inches='tight')

In [ ]:
# Plot of daily cases
plot_train_test_fore(train_original.Confirmed, test_original[['undiff_Confirmed']], fore_original[['undiff_Confirmed']], title='Daily cases', figpath='../../figures/v_ar/usa_cases.eps')



In [ ]:
# Plot of daily doses
# plot_train_test_fore(train_original.Total_Doses, test_original[['undiff_Total_Doses']], fore_original[['undiff_Total_Doses']], title='Daily doses', figpath='../../figures/v_ar/usa_vacc.eps')




### VAR(10) Model

In [ ]:
model_10 = VARMAX(train, order=(10,0))
model_fit_10 = model_10.fit()
forecasted_10 = model_fit_10.forecast(len(test))

df_forecast_10 = pd.DataFrame(forecasted_10, index=test.index)
df_forecast_10.rename(columns = {'Confirmed': 'Confirmed_forecast', 'Total_Doses': 'Total_Doses_forecast'}, inplace=True)

forecasted_conf_10 = df_forecast_10.join(test)[['Confirmed', 'Confirmed_forecast']]
forecasted_vacc_10 = df_forecast_10.join(test)[['Total_Doses', 'Total_Doses_forecast']]

In [ ]:
model_fit_10.summary()

### Plot Predictions on Diffed

In [ ]:
# plot_fore_test(test[['Confirmed']], forecasted_10[['Confirmed']], title='Diffed Daily cases')

In [ ]:
# plot_fore_test(test[['Total_Doses']], forecasted_10[['Total_Doses']], title='Diffed Daily vaccinations')

## Un-differencing and Plotting

In [ ]:
renamed_df_10 = df_forecast_10.rename(columns={'Confirmed_forecast': 'Confirmed', 'Total_Doses_forecast': 'Total_Doses'}, inplace=False)


start_index = indexed.index.get_loc(renamed_df_10.index[0])-1
fore_original_10 = invert_transformation(renamed_df_10, indexed.iloc[start_index:])

### Store metrics

In [ ]:
results_table = results_table.append({
    'model': 'VAR(10)',
    'mse': mean_squared_error(test_original['undiff_Confirmed'], fore_original_10['undiff_Confirmed']),
    'mape': MAPE(test_original['undiff_Confirmed'], fore_original_10['undiff_Confirmed']),
    'mae': mean_absolute_error(test_original['undiff_Confirmed'], fore_original_10['undiff_Confirmed'])
}, ignore_index=True)

results_table

In [ ]:
# Plot of daily cases
plot_train_test_fore(train_original.Confirmed, test_original[['undiff_Confirmed']], fore_original_10[['undiff_Confirmed']], title='Daily cases')



## VMA Model

In [ ]:
acf_varma_confirmed = plot_acf(train['Confirmed'], lags=25)

Possible values of q - 2, 6, 7, 9, 13, 14

In [ ]:
evaluate_models(train, test, [0], [2, 6, 7], column='Confirmed')

## VMA(2)

In [ ]:
model_ma2 = VARMAX(train, order=(0,2))
model_fit_ma2 = model_ma2.fit()
forecasted_ma2 = model_fit_ma2.forecast(len(test))

df_forecast_ma2 = pd.DataFrame(forecasted_ma2, index=test.index)
df_forecast_ma2.rename(columns = {'Confirmed': 'Confirmed_forecast', 'Total_Doses': 'Total_Doses_forecast'}, inplace=True)

forecasted_conf_ma2 = df_forecast_ma2.join(test)[['Confirmed', 'Confirmed_forecast']]
forecasted_vacc_ma2 = df_forecast_ma2.join(test)[['Total_Doses', 'Total_Doses_forecast']]

In [ ]:
model_fit_ma2.summary()

In [ ]:
# plot_fore_test(test[['Confirmed']], forecasted_ma2[['Confirmed']], title='Diffed Daily cases')

In [ ]:
# plot_fore_test(test[['Total_Doses']], forecasted_ma2[['Total_Doses']], title='Diffed Daily vaccinations')

### Undifferencing and plotting

In [ ]:
renamed_df_ma2 = df_forecast_ma2.rename(columns={'Confirmed_forecast': 'Confirmed', 'Total_Doses_forecast': 'Total_Doses'}, inplace=False)


start_index_ma2 = indexed.index.get_loc(renamed_df_ma2.index[0])-1
fore_original_ma2 = invert_transformation(renamed_df_ma2, indexed.iloc[start_index_ma2:])

In [ ]:
# fore_original_ma2

In [ ]:
results_table = results_table.append({
    'model': 'VMA(2)',
    'mse': mean_squared_error(test_original['undiff_Confirmed'], fore_original_ma2['undiff_Confirmed']),
    'mape': MAPE(test_original['undiff_Confirmed'], fore_original_ma2['undiff_Confirmed']),
    'mae': mean_absolute_error(test_original['undiff_Confirmed'], fore_original_ma2['undiff_Confirmed'])
}, ignore_index=True)

results_table

In [ ]:
plot_train_test_fore(train_original.Confirmed, test_original[['undiff_Confirmed']], fore_original_ma2[['undiff_Confirmed']], title='Daily cases')



## VMA(6)

In [ ]:
model_ma6 = VARMAX(train, order=(0,6))
model_fit_ma6 = model_ma6.fit()
forecasted_ma6 = model_fit_ma6.forecast(len(test))

df_forecast_ma6 = pd.DataFrame(forecasted_ma6, index=test.index)
df_forecast_ma6.rename(columns = {'Confirmed': 'Confirmed_forecast', 'Total_Doses': 'Total_Doses_forecast'}, inplace=True)

forecasted_conf_ma6 = df_forecast_ma6.join(test)[['Confirmed', 'Confirmed_forecast']]
forecasted_vacc_ma6 = df_forecast_ma6.join(test)[['Total_Doses', 'Total_Doses_forecast']]

In [ ]:
model_fit_ma6.summary()

In [ ]:
# plot_fore_test(test[['Confirmed']], forecasted_ma6[['Confirmed']], title='Diffed Daily cases')

In [ ]:
# plot_fore_test(test[['Total_Doses']], forecasted_ma6[['Total_Doses']], title='Diffed Daily vaccinations')

In [ ]:
renamed_df_ma6 = df_forecast_ma6.rename(columns={'Confirmed_forecast': 'Confirmed', 'Total_Doses_forecast': 'Total_Doses'}, inplace=False)


start_index_ma6 = indexed.index.get_loc(renamed_df_ma6.index[0])-1
fore_original_ma6 = invert_transformation(renamed_df_ma6, indexed.iloc[start_index_ma6:])

In [ ]:
# fore_original_ma6

In [ ]:
results_table = results_table.append({
    'model': 'VMA(6)',
    'mse': mean_squared_error(test_original['undiff_Confirmed'], fore_original_ma6['undiff_Confirmed']),
    'mape': MAPE(test_original['undiff_Confirmed'], fore_original_ma6['undiff_Confirmed']),
    'mae': mean_absolute_error(test_original['undiff_Confirmed'], fore_original_ma6['undiff_Confirmed'])
}, ignore_index=True)

results_table

In [ ]:
plot_train_test_fore(train_original.Confirmed, test_original[['undiff_Confirmed']], fore_original_ma6[['undiff_Confirmed']], title='Daily cases')



## VMA(7)

In [ ]:
model_ma7 = VARMAX(train, order=(0,7))
model_fit_ma7 = model_ma7.fit()
forecasted_ma7 = model_fit_ma7.forecast(len(test))

df_forecast_ma7 = pd.DataFrame(forecasted_ma7, index=test.index)
df_forecast_ma7.rename(columns = {'Confirmed': 'Confirmed_forecast', 'Total_Doses': 'Total_Doses_forecast'}, inplace=True)

forecasted_conf_ma7 = df_forecast_ma7.join(test)[['Confirmed', 'Confirmed_forecast']]
forecasted_vacc_ma7 = df_forecast_ma7.join(test)[['Total_Doses', 'Total_Doses_forecast']]

In [ ]:
model_fit_ma7.summary()

In [ ]:
# plot_fore_test(test[['Confirmed']], forecasted_ma7[['Confirmed']], title='Diffed Daily cases')

In [ ]:
# plot_fore_test(test[['Total_Doses']], forecasted_ma7[['Total_Doses']], title='Diffed Daily vaccinations')

In [ ]:
renamed_df_ma7 = df_forecast_ma7.rename(columns={'Confirmed_forecast': 'Confirmed', 'Total_Doses_forecast': 'Total_Doses'}, inplace=False)


start_index_ma7 = indexed.index.get_loc(renamed_df_ma7.index[0])-1
fore_original_ma7 = invert_transformation(renamed_df_ma7, indexed.iloc[start_index_ma7:])

In [ ]:
# fore_original_ma7

In [ ]:
results_table = results_table.append({
    'model': 'VMA(7)',
    'mse': mean_squared_error(test_original['undiff_Confirmed'], fore_original_ma7['undiff_Confirmed']),
    'mape': MAPE(test_original['undiff_Confirmed'], fore_original_ma7['undiff_Confirmed']),
    'mae': mean_absolute_error(test_original['undiff_Confirmed'], fore_original_ma7['undiff_Confirmed'])
}, ignore_index=True)

results_table

In [ ]:
plot_train_test_fore(train_original.Confirmed, test_original[['undiff_Confirmed']], fore_original_ma7[['undiff_Confirmed']], title='Daily cases')



<a name=varma></a>

## VARMA

In [ ]:
pacf_varma_confirmed = plot_pacf(train['Confirmed'], lags=25)
acf_varma_confirmed = plot_acf(train['Confirmed'], lags=25)

In [ ]:
# evaluate_models(train, test, [3, 10], [2, 6, 7], column='Confirmed')

In [ ]:
model_varma = VARMAX(train, order=(3,2))
model_fit_varma = model_varma.fit()
yhat_varma = model_fit_varma.forecast(len(test))
#yhat

In [ ]:
# plot_fore_test(test[['Confirmed']], yhat_varma[['Confirmed']], title='Diffed Daily cases')

In [ ]:
# plot_fore_test(test[['Total_Doses']], yhat_varma[['Total_Doses']], title='Diffed Daily Doses')

In [ ]:
# Un-diff the test dataset

fore_original_varma = invert_transformation(yhat_varma, indexed.iloc[start_index:])
fore_original_varma

In [ ]:
# plot_fore_test(test_original[['undiff_Confirmed']], fore_original_varma[['undiff_Confirmed']], title='Daily cases')

In [ ]:
# plot_fore_test(test_original[['undiff_Total_Doses']], fore_original_varma[['undiff_Total_Doses']], title='Daily Doses')

In [ ]:
# plot_subplots([train_original, fore_original_varma])

In [ ]:
plot_train_test_fore(train_original.Confirmed, test_original[['undiff_Confirmed']], fore_original_varma[['undiff_Confirmed']], title='Daily cases')



In [ ]:
results_table = results_table.append({
    'model': 'VARMA(3,2)',
    'mse': mean_squared_error(test_original['undiff_Confirmed'], fore_original_varma['undiff_Confirmed']),
    'mape': MAPE(test_original['undiff_Confirmed'], fore_original_varma['undiff_Confirmed']),
    'mae': mean_absolute_error(test_original['undiff_Confirmed'], fore_original_varma['undiff_Confirmed'])
}, ignore_index=True)

results_table

<a name=roll2></a>

## Rolling Forecasts

In [ ]:
history = train.copy()
predicted = pd.DataFrame(columns=[
    'VAR_Confirmed', 
    'VAR_Total_Doses', 
    'VMA_Confirmed', 
    'VMA_Total_Doses', 
    'VARMA_Confirmed', 
    'VARMA_Total_Doses'
], index=test.index)


# predicted

for t in range(len(test)):
    # 3 models
    var = VARMAX(history, order=(3,0))
    vma = VARMAX(history, order=(0,2))
    varma = VARMAX(history, order=(3,2))
    
    # 3 fitted models
    var_fit = var.fit()
    vma_fit = vma.fit()
    varma_fit = varma.fit()
    
    # Predictions
    yhat_var = var_fit.forecast()
    yhat_vma = vma_fit.forecast()
    yhat_varma = varma_fit.forecast()
    
    # Next index to insert
    newindex = history.index[-1] + pd.to_timedelta(1, 'D')
    
    # Confirmed cases
    predicted.loc[newindex]['VAR_Confirmed'] = yhat_var['Confirmed'].values[0]
    predicted.loc[newindex]['VMA_Confirmed'] = yhat_vma['Confirmed'].values[0]
    predicted.loc[newindex]['VARMA_Confirmed'] = yhat_varma['Confirmed'].values[0]
    
    # Total doses
    predicted.loc[newindex]['VAR_Total_Doses'] = yhat_var['Total_Doses'].values[0]
    predicted.loc[newindex]['VMA_Total_Doses'] = yhat_vma['Total_Doses'].values[0]
    predicted.loc[newindex]['VARMA_Total_Doses'] = yhat_varma['Total_Doses'].values[0]
    
    history = history.append(test.iloc[t])
    # history[newindex]['Total_Doses'] = test.iloc[t]['Total_Doses']
    
    # print('predicted =', yhat.values, ' ; actual =', test.iloc[t].values)

In [ ]:
predicted

## VAR(3)

In [ ]:
plot_subplots([test, predicted[['VAR_Confirmed', 'VAR_Total_Doses']]])

In [ ]:
# Un-diffing

pred_roll_var = predicted[['VAR_Confirmed', 'VAR_Total_Doses']].rename(columns={'VAR_Confirmed': 'Confirmed', 'VAR_Total_Doses': 'Total_Doses'}, inplace=False)
fore_original_roll_var = invert_transformation(pred_roll_var, indexed.iloc[start_index:])

In [ ]:
results_table = results_table.append({
    'model': 'VAR(3) - rolling',
    'mse': mean_squared_error(test_original['undiff_Confirmed'], fore_original_roll_var['undiff_Confirmed']),
    'mape': MAPE(test_original['undiff_Confirmed'], fore_original_roll_var['undiff_Confirmed']),
    'mae': mean_absolute_error(test_original['undiff_Confirmed'], fore_original_roll_var['undiff_Confirmed'])
}, ignore_index=True)

results_table

In [ ]:
# fore_original_roll_var

In [ ]:
plot_train_test_fore(train_original.Confirmed, test_original[['undiff_Confirmed']], fore_original_roll_var[['undiff_Confirmed']], title='Daily cases')




## VMA(2)

In [ ]:
plot_subplots([test, predicted[['VMA_Confirmed', 'VMA_Total_Doses']]])

In [ ]:
# Un-diffing

pred_roll_vma = predicted[['VMA_Confirmed', 'VMA_Total_Doses']].rename(columns={'VMA_Confirmed': 'Confirmed', 'VMA_Total_Doses': 'Total_Doses'}, inplace=False)
fore_original_roll_vma = invert_transformation(pred_roll_vma, indexed.iloc[start_index:])

In [ ]:
results_table = results_table.append({
    'model': 'VMA(2) - rolling',
    'mse': mean_squared_error(test_original['undiff_Confirmed'], fore_original_roll_vma['undiff_Confirmed']),
    'mape': MAPE(test_original['undiff_Confirmed'], fore_original_roll_vma['undiff_Confirmed']),
    'mae': mean_absolute_error(test_original['undiff_Confirmed'], fore_original_roll_vma['undiff_Confirmed'])
}, ignore_index=True)

results_table

In [ ]:
# fore_original_roll_vma

In [ ]:
plot_train_test_fore(train_original.Confirmed, test_original[['undiff_Confirmed']], fore_original_roll_vma[['undiff_Confirmed']], title='Daily cases')




## VARMA(3,2)

In [ ]:
plot_subplots([test, predicted[['VARMA_Confirmed', 'VARMA_Total_Doses']]])

In [ ]:
# Un-diffing

pred_roll_varma = predicted[['VARMA_Confirmed', 'VARMA_Total_Doses']].rename(columns={'VARMA_Confirmed': 'Confirmed', 'VARMA_Total_Doses': 'Total_Doses'}, inplace=False)
fore_original_roll_varma = invert_transformation(pred_roll_varma, indexed.iloc[start_index:])

In [ ]:
results_table = results_table.append({
    'model': 'VARMA(3,2) - rolling',
    'mse': mean_squared_error(test_original['undiff_Confirmed'], fore_original_roll_varma['undiff_Confirmed']),
    'mape': MAPE(test_original['undiff_Confirmed'], fore_original_roll_varma['undiff_Confirmed']),
    'mae': mean_absolute_error(test_original['undiff_Confirmed'], fore_original_roll_varma['undiff_Confirmed'])
}, ignore_index=True)

results_table

In [ ]:
# fore_original_roll_varma

In [ ]:
plot_train_test_fore(train_original.Confirmed, test_original[['undiff_Confirmed']], fore_original_roll_varma[['undiff_Confirmed']], title='Daily cases')




In [ ]:
results_table